In [1]:
from graph_reciprocity import *
from nonlinear_selection import *
from graph_epidemic import *
%load_ext autoreload
%autoreload 2

In [2]:
def get_y_eff(y,k):
    return y*(1 + (1-y)/(y*k))

def get_s_eff(y,alpha,beta,k):
    return alpha*get_y_eff(y,k) - beta

In [3]:
n_n = 5
c_r = 0.5
N = 20
beta = 4.0/(c_r*n_n)
alpha = (N*beta)/n_n

k_sparse = 4.0
k = N-1
print N,alpha,beta
print (beta/alpha - 1.0/k_sparse)/(1 - 1.0/k_sparse)

y_n, y_minus,y_plus,y_p,critical_determinant = get_parameters(N,alpha,beta)

20 6.4 1.6
0.0
y_n = 0.25, y_- = 0.0366116523517, y_+ = 0.213388347648, y_p = 0.278093108924, critical determinant = 0.5
n_n = 5.0


In [4]:
#%pylab inline
plot_schematic(n_n,c_r,N,4)

nonlinear_selection.py:125: RuntimeWarning: invalid value encountered in divide
  return f(y)/y - beta
nonlinear_selection.py:128: RuntimeWarning: divide by zero encountered in divide
  return y*(1 + (1-y)/(y*k))


20 6.4 1.6
y_n = 0.25, y_- = 0.0366116523517, y_+ = 0.213388347648, y_p = 0.278093108924, critical determinant = 0.5
n_n = 5.0


nonlinear_selection.py:128: RuntimeWarning: invalid value encountered in multiply
  return y*(1 + (1-y)/(y*k))


In [6]:
k = N- 1
k_sparse = 4.0
num_trials_well_mixed = 20000
num_trials = 1000
regular = True
plotting = False
epidemic_sizes,fixed = run_epidemics(N,alpha,beta,num_trials=num_trials_well_mixed,plotting=plotting)
epidemic_sizes_g, fixed_g = run_epidemics(N,alpha,beta,num_trials=num_trials,plotting=plotting,\
                                      trajectory_fn = lambda a,b,c: simulate_graph_trajectory_adaptive(a,b,c,k=k,regular= regular))
# epidemic_sizes_g2, fixed_g2 = run_epidemics(N,alpha,beta,num_trials=num_trials,plotting=plotting,\
#                                       trajectory_fn = lambda a,b,c: simulate_graph_trajectory_adaptive(a,b,c,k=k/2.0))
# epidemic_sizes_g5, fixed_g5 = run_epidemics(N,alpha,beta,num_trials=num_trials,plotting=plotting,\
#                                       trajectory_fn = lambda a,b,c: simulate_graph_trajectory_adaptive(a,b,c,k=k_sparse,regular=regular))

#print (1.0/N)/y_p
print P_fix(1.0/N,alpha,beta,N)

y_n = 0.25, y_- = 0.0366116523517, y_+ = 0.213388347648, y_p = 0.278093108924, critical determinant = 0.5
n_n = 5.0
T_ave = 0.566472421223, P_fix = 0.02735
y_n = 0.25, y_- = 0.0366116523517, y_+ = 0.213388347648, y_p = 0.278093108924, critical determinant = 0.5
n_n = 5.0
T_ave = 0.552756756757, P_fix = 0.0
0.0309822038632


In [11]:
sum(fixed_g)/len(fixed_g)

0.0

In [16]:
e_sizes_arr = [epidemic_sizes,epidemic_sizes_g,epidemic_sizes_g2,epidemic_sizes_g5]
fixed_arr = [fixed,fixed_g,fixed_g2,fixed_g5]

In [32]:
savez('N40run',N=N,alpha=alpha,beta=beta,k_sparse=k_sparse,e_sizes_arr=e_sizes_arr,fixed_arr=fixed_arr)


In [9]:
data = load('N40run.npz')
e_sizes_arr = data['e_sizes_arr']
fixed_arr = data['fixed_arr']
close(3)
for i,e_sizes in enumerate(e_sizes_arr):
    print mean(e_sizes)
    if i ==2:
        continue

    fixed_curr = fixed_arr[i]

    #close(1)
    figure(3)
    hold(1)
    nbins = 40
    bins = np.logspace(0,log10(max(e_sizes[fixed_curr == 0])),nbins)
    hist(e_sizes[fixed_curr == 0],log=True,bins=bins,alpha=0.3,normed=True,label='simulation' + str(i))
    xlabel('$w$',size=20)
    ylabel('$P(w)$',size=20)
    gca().set_xscale('log')
    w_range = np.logspace(0,log10(max(e_sizes[fixed_curr == 0])))
    P_w_th_range = P_w_th(w_range,s(sqrt(w_range)/N,alpha,beta))
    P_w_th_range_eff = P_w_th(w_range,get_s_eff(sqrt(w_range)/N,alpha,beta,k_sparse))

    def normed_distribution(x,px):
        return px/sum(diff(x)*px[:-1])

    if i == 0:
        #normed = integrate.quad(lambda x: P_w_th(x,0),min(epidemic_sizes[fixed_curr==0]),max(epidemic_sizes[fixed_curr==0]))[0]
        plot(w_range,normed_distribution(w_range,P_w_th_range),'-r',label=r'theory')#$P(w) \sim e^{- s(\sqrt{w})^2 w/4} w^{-3/2}/(1 + s(\sqrt{w}))$ (theory)')
        plot(w_range,normed_distribution(w_range,P_w_th_range_eff),'-g',label=r'effective theory')#$P(w) \sim e^{- s(\sqrt{w})^2 w/4} w^{-3/2}/(1 + s(\sqrt{w}))$ (theory)')

        #plot(w_range,s(sqrt(w_range)/N,alpha,beta)**2*w_range/4.0,label=r'$s(\sqrt{w})^2 w /4$')
        axvline((2*y_p*N)**2,color = 'k',label=r'$w=4(y_p N)^2$')
        axvline((2*y_n*N)**2,color = 'k',label=r'$w=4(y_n N)^2$',linestyle='--')
        grid()
        if (y_minus > 0):
            axvline((2*y_minus*N)**2,color = 'r',label=r'$w=4(y_- N)^2$')
    legend(prop={'size':15},loc='upper right')

1.81508431144
1.32000595238
1.4420297619
1.63073690476


In [5]:
%load_ext line_profiler
num_trials = 20
regular = True
plotting = False

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [10]:
%lprun -f update_node -f get_infected_neighbor_fraction run_epidemics(N,alpha,beta,num_trials=num_trials,plotting=plotting,\
                                       trajectory_fn = lambda a,b,c: simulate_graph_trajectory_adaptive(a,b,c,k=N-1,regular= regular))


y_n = 0.1, y_- = 0.0146446609407, y_+ = 0.0853553390593, y_p = 0.11123724357, critical determinant = 0.5
n_n = 10.0
T_ave = 0.384444444444, P_fix = 0.0



In [16]:
a

0.0